In [1]:
!pip install yt-dlp pydub

!apt-get install ffmpeg -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import os
from pydub import AudioSegment
from yt_dlp import YoutubeDL

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [3]:
DOWNLOAD_FOLDER = "/content/raw_audio"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

# Folder to save 3-10 sec clips
CLIP_FOLDER = "/content/calm_audio_clips"
os.makedirs(CLIP_FOLDER, exist_ok=True)

In [4]:
MIN_SEC = 3
MAX_SEC = 10

In [7]:
youtube_urls = [
    "https://www.youtube.com/watch?v=gisytJ0n2jI",
    'https://www.youtube.com/watch?v=9EqrUK7ghho&t=130s'
]


In [8]:
def download_audio(youtube_urls, output_folder):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': False
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download(youtube_urls)

In [10]:
def split_audio_clips(source_folder, clip_folder, min_sec=3, max_sec=10):
    for file in os.listdir(source_folder):
        if file.endswith(".mp3") or file.endswith(".wav"):
            audio_path = os.path.join(source_folder, file)
            try:
                audio = AudioSegment.from_file(audio_path)
                duration_sec = len(audio) / 1000  # milliseconds to seconds
                if duration_sec < min_sec:
                    continue  # skip too short
                # If longer than max_sec, split into multiple clips
                start = 0
                clip_index = 0
                while start < len(audio):
                    end = start + max_sec*1000
                    clip = audio[start:end]
                    clip_length_sec = len(clip)/1000
                    if clip_length_sec >= min_sec:
                        clip_name = f"{os.path.splitext(file)[0]}_clip{clip_index}.mp3"
                        clip.export(os.path.join(clip_folder, clip_name), format="mp3")
                        clip_index += 1
                    start += max_sec*1000
            except Exception as e:
                print(f"Error processing {file}: {e}")

In [11]:
print("Downloading YouTube audios...")
download_audio(youtube_urls, DOWNLOAD_FOLDER)

[youtube] Extracting URL: https://www.youtube.com/watch?v=gisytJ0n2jI
[youtube] gisytJ0n2jI: Downloading webpage


[youtube] gisytJ0n2jI: Downloading android sdkless player API JSON
[youtube] gisytJ0n2jI: Downloading web safari player API JSON


[youtube] gisytJ0n2jI: Downloading m3u8 information


[info] gisytJ0n2jI: Downloading 1 format(s): 251
[download] Sleeping 6.00 seconds as required by the site...
[download] Destination: /content/raw_audio/The Most Relaxing Psychology Facts to Fall Asleep To — Dreamy Science.webm
[download] 100% of  118.15MiB in 00:00:06 at 17.71MiB/s  
[ExtractAudio] Destination: /content/raw_audio/The Most Relaxing Psychology Facts to Fall Asleep To — Dreamy Science.mp3
Deleting original file /content/raw_audio/The Most Relaxing Psychology Facts to Fall Asleep To — Dreamy Science.webm (pass -k to keep)
[youtube] Extracting URL: https://www.youtube.com/watch?v=9EqrUK7ghho&t=130s
[youtube] 9EqrUK7ghho: Downloading webpage


[youtube] 9EqrUK7ghho: Downloading android sdkless player API JSON
[youtube] 9EqrUK7ghho: Downloading web safari player API JSON


[youtube] 9EqrUK7ghho: Downloading m3u8 information


[info] 9EqrUK7ghho: Downloading 1 format(s): 251
[download] Sleeping 6.00 seconds as required by the site...
[download] Destination: /content/raw_audio/The Stress Expert： Your Brain is Like a Phone Battery! (9 Ways to Instantly Recharge).webm
[download] 100% of   69.72MiB in 00:00:03 at 18.76MiB/s  
[ExtractAudio] Destination: /content/raw_audio/The Stress Expert： Your Brain is Like a Phone Battery! (9 Ways to Instantly Recharge).mp3
Deleting original file /content/raw_audio/The Stress Expert： Your Brain is Like a Phone Battery! (9 Ways to Instantly Recharge).webm (pass -k to keep)


In [12]:
print("Splitting into 3-10 second clips...")
split_audio_clips(DOWNLOAD_FOLDER, CLIP_FOLDER, MIN_SEC, MAX_SEC)


Splitting into 3-10 second clips...


In [13]:
import shutil

# Replace with your folder path
shutil.make_archive("/content/calm_audio_clips_zip", 'zip', "/content/calm_audio_clips")


'/content/calm_audio_clips_zip.zip'

In [14]:
from google.colab import files

files.download("/content/calm_audio_clips_zip.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>